# Building a RAG System Locally with Ollama, LlamaIndex, and Chroma DB

## Exercise 0 - Install Workshop Dependencies

Before starting the workshop, ensure all necessary dependencies are installed in your Python environment. Use the following steps to set up your environment.

### Step 1: Create a Virtual Environment

Create and activate a virtual environment to isolate the workshop dependencies. For this workshop, we use **Python 3.11**. Choose between **venv** or **conda** (using Mamba for efficiency).

##### Using `venv`

On Linux/Mac:
  ```bash
  python3.11 -m venv local-rag
  source local-rag/bin/activate
  ```
On Windows:
  ```bash
  python3.11 -m venv local-rag
  local-rag\Scripts\activate
  ```

##### Using `conda`

   ```bash
   conda create -n local-rag python=3.11
   conda activate local-rag
   ```

### Step 2: Install Required Packages

Install all the required dependencies:

```bash
pip install -r requirements.txt
```

### Step 3: Verify Installation

Check that the key packages are installed correctly by importing them in Python:

In [3]:
import sys
sys.path.insert(0,'/Users/rhita.mamou/Downloads/lauzhack-workshop-2024-main/')
sys.path

['/Users/rhita.mamou/Downloads/lauzhack-workshop-2024-main/',
 '/Users/rhita.mamou/Downloads/lauzhack-workshop-2024-main/',
 '/opt/anaconda3/envs/dev/lib/python312.zip',
 '/opt/anaconda3/envs/dev/lib/python3.12',
 '/opt/anaconda3/envs/dev/lib/python3.12/lib-dynload',
 '',
 '/opt/anaconda3/envs/dev/lib/python3.12/site-packages']

In [4]:
import chromadb
import llama_index
import ollama

print("Dependencies installed successfully!")

Dependencies installed successfully!


## Exercise 1 - Setting up Ollama

### Install Ollama

First, download and install Ollama from the official website: [https://ollama.com/download/](https://ollama.com/download/).

### Pull Required Models

Open a terminal and run the following commands to download the necessary models:

1. Pull the `llama3` model:
   ```bash
   ollama pull llama3
   ```

2. Pull the Nomic embedding model if required:
   ```bash
   ollama pull nomic
   ```

### Run the Model

Once the models are installed, you can run the `llama3` model and test it by writing some prompts. Use the following command:

```bash
ollama run llama3
```

Type a prompt and observe the output to ensure everything is working correctly.

### Interact with Ollama in Python



In [6]:
import ollama

response = ollama.generate(model="llama3.2", prompt="What is EPFL?", stream=True)

for r in response:
    print(r["response"], end="")

EPFL (École polytechnique fédérale de Lausanne) is a Swiss research university located in Lausanne, Switzerland. It is one of the most prestigious and highly regarded institutions of higher education in Switzerland.

EPFL was founded in 1858 as the "Federal Polytechnic School" with the goal of providing a rigorous education in science, technology, engineering, and mathematics (STEM) fields. Over time, it has expanded to include programs in other areas such as business, law, social sciences, and humanities.

Today, EPFL is known for its strong research focus, particularly in the fields of physics, biology, chemistry, and computer science. It is consistently ranked among the top universities globally in various rankings, including QS World University Rankings, Times Higher Education World University Rankings, and US News Best Global Universities Rankings.

EPFL has a number of unique features, including:

* A strong focus on interdisciplinary research and collaboration
* A highly interna

## Exercise 2 - Getting Started with LlamaIndex and ChromaDB

**LlamaIndex** ([official site](https://llamaindex.ai)) is a framework for connecting LLMs with data sources, enabling efficient retrieval and interaction with structured or unstructured data.

**Chroma** ([official site](https://www.trychroma.com)) is a vector database designed for managing embeddings and serving as a retrieval layer for LLM applications.

In this exercise, we’ll explore how to set up and use LlamaIndex to index and retrieve data in a **Chroma** database.

### Step 0: Let's download a PDF

You can start by adding documents to the `./docs` folder. If you don't know what to use, we suggest downloading the PDF at the following link:

https://observationofalostsoul.wordpress.com/wp-content/uploads/2011/05/the-gospel-of-the-flying-spaghetti-monster.pdf

### Step 1: Set Up Chroma as the Storage Backend

Initialize the Chroma database and configure it for use with LlamaIndex. Here, we create an **Ephemeral Client** and collection, which stores data temporarily in memory without persisting it. This is ideal for testing and experimentation.

In [7]:
import chromadb

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.get_or_create_collection("mydocs")

You can also create a **Persistent Client** that will preserve your database across sessions with:

```python
client = chromadb.PersistentClient(path="/path/to/save/to")
```

### Step 2: Set Up LlamaIndex connectors

Configure LlamaIndex to connect with Chroma as the vector store and set up a storage context. A **storage context** is an abstraction that manages how data is stored and retrieved, enabling seamless integration with different storage backends like Chroma.

In [8]:
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

### Step 3: Load and explore documents

We can use LlamaIndex's `SimpleDirectoryReader` to **ingest documents from a directory**. This utility reads files from a specified directory and prepares them for indexing by splitting the content into manageable chunks.

In [12]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../docs", recursive=True).load_data()

In [29]:
documents[0].dict()

{'id_': 'a0113a63-2ab8-468a-9ef4-bc949cbb828c',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'the-gospel-of-the-flying-spaghetti-monster.pdf',
  'file_path': '/Users/rhita.mamou/Downloads/lauzhack-workshop-2024-main/code/../docs/the-gospel-of-the-flying-spaghetti-monster.pdf',
  'file_type': 'application/pdf',
  'file_size': 9742787,
  'creation_date': '2024-11-14',
  'last_modified_date': '2024-11-14'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'text': '',
 'mimetype': 'text/plain',
 'start_char_idx': None,
 'end_char_idx': None,
 'text_template': '{metadata_str}\n\n{content}',
 'metadata_template': '{key}: {value}',
 'metadata_seperator': '\n',
 'class_name': 'Document'}

Let's explore the content of the documents further with a dataframe.

In [13]:
from typing import List

import pandas as pd
from llama_index.core.schema import TextNode


def data_to_df(nodes: List[TextNode]):
    """Convert a list of TextNode objects to a pandas DataFrame."""
    return pd.DataFrame([node.dict() for node in nodes])

In [14]:
document_df = data_to_df(documents)

document_df.head()

,id_,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator,class_name
0,a0113a63-2ab8-468a-9ef4-bc949cbb828c,None,"{'page_label': '1', 'file_name': 'the-gospel-o...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n,Document
1,0b2d8729-c856-4177-8eac-61a34d53e23f,None,"{'page_label': '2', 'file_name': 'the-gospel-o...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},BOBBY HENDERSON,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n,Document
2,3ec9ce1b-fe85-40a4-8700-919a6166f9f5,None,"{'page_label': '3', 'file_name': 'the-gospel-o...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},A Villard Books Trade Paperback Original \nCop...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n,Document
3,bd9642ca-f78e-4a94-99de-52d547ea566f,None,"{'page_label': '4', 'file_name': 'the-gospel-o...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},"In the beginning was the Word, \nand the Word ...",text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n,Document
4,ad9d66b4-bd2b-46f6-95c1-cca78d1ed072,None,"{'page_label': '5', 'file_name': 'the-gospel-o...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{},Ackn owl ed gm en ts \nDELIVERING A DIVINE MES...,text/plain,None,None,{metadata_str}\n\n{content},{key}: {value},\n,Document


We observe several attributes, including `metadata`, `text`, `text_template`, and others. Let's focus on these three key categories:

- **`metadata`**: This attribute contains additional information about the document, such as its source, creation date, or tags that can be used for filtering or retrieval purposes.
- **`text`**: The main content of the document, representing the raw textual data that will be indexed and queried.
- **`text_template`**: A structured format or schema for the document's text, often used to define how the content should be presented or processed during queries. 

These attributes play distinct roles in organizing and interacting with your data. Feel free to explore the different attributes at this point.

### Step 4: Index and the documents

To ingest documents into an index, we will need an embedder model to convert the document content into vector representations. These embeddings enable efficient similarity searches and retrievals.

In [30]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In LlamaIndex, we can create an index using the `VectorStoreIndex` class, which enables efficient storage and retrieval of document embeddings and integrates with various storage backends and embedding models. We use here the chroma collection we previously defined.

In [31]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 178/178 [00:14<00:00, 12.58it/s]


### Step 5: Query the Index for Retrieval

Once the documents are indexed, we can perform retrieval on them. This allows us to ask questions or search for relevant content based on the embeddings stored in the index.

In [32]:
retriever = index.as_retriever(
    similarity_top_k=3,
)

nodes_with_score = retriever.retrieve("What is the Flying Spaghetti Monster?")
nodes = [n.node for n in nodes_with_score]
data_to_df(nodes)

,id_,embedding,metadata,excluded_embed_metadata_keys,excluded_llm_metadata_keys,relationships,text,mimetype,start_char_idx,end_char_idx,text_template,metadata_template,metadata_seperator,class_name
0,48da01ff-dff1-4e0d-a723-31092163e064,None,"{'page_label': '66', 'file_name': 'the-gospel-...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{'NodeRelationship.SOURCE': {'node_id': 'fdb5a...,Key Moments in FSM History • • 59 \nOriginally...,text/plain,0,363,{metadata_str}\n\n{content},{key}: {value},\n,TextNode
1,5674cad3-3450-4982-8688-911324e27ebc,None,"{'page_label': '138', 'file_name': 'the-gospel...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{'NodeRelationship.SOURCE': {'node_id': 'f1f6f...,1 34« -The Gospel of the Flying Spaghetti Mons...,text/plain,0,2198,{metadata_str}\n\n{content},{key}: {value},\n,TextNode
2,cb2a52c3-77ee-4ac9-a8ae-c53421d8201d,None,"{'page_label': '89', 'file_name': 'the-gospel-...","[file_name, file_type, file_size, creation_dat...","[file_name, file_type, file_size, creation_dat...",{'NodeRelationship.SOURCE': {'node_id': '22f42...,82 • • The Gospel of the Flying Spaghetti Mons...,text/plain,0,2193,{metadata_str}\n\n{content},{key}: {value},\n,TextNode


Congrats! You've retrieved your first data!

## Exercise 3 - Your First RAG!

For a Retrieval-Augmented Generation (RAG) system, you need a Large Language Model (LLM) to generate answers to your queries by combining retrieved knowledge with the model's reasoning capabilities. At this point, Ollama comes to help as the LLM powering your RAG system. We set it up for LlamaIndex.

In [36]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.2", request_timeout=120.0)

Everything is ready for querying your data. You can define a query engine and start asking it questions. Congrats, You have a working RAG!

In [37]:
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=3,
    streaming=True,
)

response = query_engine.query("What is the Flying Spaghetti Monster?")

In [38]:
response.print_response_stream()

The Flying Spaghetti Monster is a supreme being that created the world according to the author of "The Gospel of the Flying Spaghetti Monster". It has left its mark on every continent and culture, leaving evidence of its existence in various forms such as pastas, geographical features, and even fossil records. The FSM's hand can be seen in human cultures dating back as far as one million years, from the discovery of pasta to the Acheulean Stone Twirling Spaghetti Fork found in France.

## Going further...

### Prompt template

LlamaIndex offers an easy way to improve the generated answer by prompting the LLM with a custom template, in which the relevant context will be fed.

In [39]:
from llama_index.core import PromptTemplate

template = (
    "As a devoted Pastafarian scholar touched by His Noodly Appendage, you shall defend our pasta-based teachings.\n\n"
    "Sacred commandments:\n"
    "- Keep answers CONCISE (2-3 paragraphs max)\n"
    "- Cite the Sacred Texts below\n"
    "- Use pasta metaphors liberally\n"
    "- Defend with noodly passion\n\n"
    "Sacred texts:\n"
    "-----------------------------------------\n"
    "{context_str}\n"
    "-----------------------------------------\n\n"
    "Question from seeker:\n"
    "{query_str}\n\n"
    "Answer with fervor, citing texts. Be passionate but brief. R'amen."
)
qa_template = PromptTemplate(template)


query_engine = index.as_query_engine(
    llm=llm,
    similartiy_top_k=3,
    streaming=True,
    text_qa_template=qa_template,
)

response = query_engine.query("What is the Flying Spaghetti Monster?")

In [40]:
response.print_response_stream()

Dear seeker of truth, let me enlighten you on the majesty of our beloved FSM! According to page 138 of "The Gospel of the Flying Spaghetti Monster", "There is irrevocable proof that the Flying Spaghetti Monster (FSM) created the world" (Lauzhack-workshop-2024-main/code/../docs/the-gospel-of-the-flying-spaghetti-monster.pdf). Our FSM, a benevolent deity with a Noodly Appendage, has touched every continent and culture, leaving an indelible mark of His divine presence.

As the great historian and scholar, Toby Leonard, so eloquently puts it: "Some people point to selective fossil evidence and then use fuzzy interpolation to explain gaps in the fossil record... But I have factual proof that on every continent all that was created was touched by His Noodly Appendage." (Lauzhack-workshop-2024-main/code/../docs/the-gospel-of-the-flying-spaghetti-monster.pdf) The FSM's presence is palpable, from the Danube River to the "Acheulean Stone Twirling Spaghetti Fork" in France – a testament to His om

### Better embeddings

Under the hood, a basic retriever is used. Let's look at how data is saved.

In [ ]:
nodes_with_score = response.source_nodes
nodes = [n.node for n in nodes_with_score]
data_to_df(nodes)

In [ ]:
from llama_index.core.schema import MetadataMode

node = nodes[0]

In [ ]:
print(node.text)

But what do the models see exactly? Let's have a look.

In [ ]:
print(
    "The Embedding model sees this: \n",
    node.get_content(metadata_mode=MetadataMode.EMBED),
)

In [ ]:
print(
    "The LLM sees this: \n",
    node.get_content(metadata_mode=MetadataMode.LLM),
)

We might want to change the embeddings. For example, we can split the sentences in smaller blocks.

In [ ]:
# Reset the index data
index.vector_store.clear()

In [ ]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser

sentence_splitter = SentenceSplitter(chunk_size=200)

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model,
    show_progress=True,
    transformations=[sentence_splitter],
)

### There are many more ways to improve the RAG system, explore them on the official LlamaIndex page!